# Cryptocurrency

In [3]:
import pandas

## read files

In [86]:
inputs = pandas.read_csv('inputs.csv', header=None)
outputs = pandas.read_csv('outputs.csv', header=None)
tags = pandas.read_csv('tags.csv')
transactions = pandas.read_csv('transactions.csv', header=None)
inputs.columns = ["id", "tx_id", "sig_id", "output_id"]
outputs.columns = ["id", "tx_id", "pk_id", "value"]
tags.columns = ["type", "service", "pk_id"]
transactions.columns = ['id', "block_id"]

## 1 Basic statistic
### 1.1 Transactions

In [72]:
def answer_1_1():
    print ("------------------- 1.1 -------------------")
    print ("How many transactions were there in total?")
    print (len(transactions['id'].unique()))
    print ("Of these, how many transactions had one input and two outputs?")
    count_outputs = outputs['tx_id'].value_counts()
    count_inputs = inputs['tx_id'].value_counts()
    count_outputs_1 = count_outputs[count_outputs == 1]
    count_outputs_2 = count_outputs[count_outputs == 2]
    count_inputs_1 = count_inputs[count_inputs == 1]
    print (len(count_outputs_2[count_outputs_2.index.isin(count_inputs_1.index)]))
    print ("How many transactions had one input and one output?")
    print (len(count_outputs_1[count_outputs_1.index.isin(count_inputs_1.index)]))

In [73]:
answer_1_1()

------------------- 1.1 -------------------
How many transactions were there in total?
216626
Of these, how many transactions had one input and two outputs?
44898
How many transactions had one input and one output?
160780


### 1.2 UTXOs

In [251]:
def answer_1_2():
    print ("------------------- 1.2 -------------------")
    temp_outputs = outputs[['id','pk_id','value']]
    temp_inputs = inputs['output_id']
    UTXOs = temp_outputs[~temp_outputs['id'].isin(temp_inputs)].set_index('id')
    print ("How many UTXOs exist, as of the last block of the dataset?")
    print (len(UTXOs))
    print ("Which UTXO has the highest associated value?")
    print ("output id:", UTXOs.value.idxmax())
    print ("value:", UTXOs.value.max())
    return UTXOs

In [252]:
UTXOs = answer_1_2()

------------------- 1.2 -------------------
How many UTXOs exist, as of the last block of the dataset?
71906
Which UTXO has the highest associated value?
output id: 170430
value: 9000000000000


### 1.3 Public keys

In [159]:
def answer_1_3():
    print ("------------------- 1.3 -------------------")
    print ("How many distinct public keys were used across all blocks in the dataset?")
    #pk_ids = pandas.concat([outputs['pk_id'], tags['pk_id']], axis=0, ignore_index=True).unique()
    print (len(outputs['pk_id'].unique())) #174702
    print ("Which public key received the highest number of bitcoins, and how many bitcoins has it received?")
    temp = outputs
    temp = temp.groupby('pk_id')['value'].sum()
    print("pk_id:", temp.idxmax(), "value:", temp.max())
    print ("Which public key acted as an output the most number of times, and how many times did it act as output?")
    count_pk = outputs['pk_id'].value_counts()
    print ("pk_id:", count_pk.idxmax(), "value:", count_pk.max())

In [10]:
answer_1_3()
# outputs.loc[outputs['pk_id']==148105]

------------------- 1.3 -------------------
How many distinct public keys were used across all blocks in the dataset?
174702
Which public key received the highest number of bitcoins, and how many bitcoins has it received?
pk: 148105 value: 27375023000000
Which public key acted as an output the most number of times, and how many times did it act as output?
pk: 148105 value: 5498


### 1.4 Invalid transactions

In [11]:
def answer_1_4():
    print ("------------------- 1.4 -------------------")
    ## double spend
    temp_spend = inputs
    temp_spend = temp_spend.loc[(temp_spend['output_id'].duplicated()) & (temp_spend['output_id'] != -1)]
    i = 0
    for _, row in temp_spend.iterrows():
        i += 1
        print (str(i)+". double spend:")
        print ("id:", row.id, "tx_id:", row.tx_id, "sig_id:", row.sig_id, "output_id:", row.output_id)
        print ("spent in:")
        s = inputs.loc[(inputs['output_id'] == row.output_id) & (inputs['id'] != row.id)]
        #print ("id:", s.id, "tx_id:", s.tx_id, "sig_id:", s.sig_id, "output_id:", s.output_id)
        row = s.reset_index().loc[0]
        print ("id:", row.id, "tx_id:", row.tx_id, "sig_id:", row.sig_id, "output_id:", row.output_id)
        print ("")
        
    ## ouput > input
    temp_input = inputs
    temp_output = outputs
    money_in = pandas.merge(temp_input[['tx_id','output_id']],temp_output[['id','value']],left_on='output_id',right_on='id')[['tx_id','value']]
    money_in = money_in.groupby('tx_id')['value'].sum()
    dict_money_in = {'tx_id':money_in.index,'money_in':money_in.values}
    money_in = pandas.DataFrame(dict_money_in)
    money_out = temp_output.groupby('tx_id')['value'].sum()
    dict_money_out = {'tx_id':money_out.index,'money_out':money_out.values}
    money_out = pandas.DataFrame(dict_money_out)
    transactions = pandas.merge(money_in[['tx_id','money_in']], money_out[['tx_id','money_out']], on='tx_id')
    invalid_tx = transactions.loc[(transactions['money_in']<transactions['money_out'])]
    for _, row in invalid_tx.iterrows():
        i += 1
        print (str(i)+". output > input:")
        print ("tx_id:", row.tx_id, "money_in:", row.money_in, "money_out:", row.money_out)
        print ("money_in:")
        j = 0
        for _, row1 in inputs.loc[inputs['tx_id']==row.tx_id].iterrows():
            for _, row2 in outputs.loc[outputs['id']==row1.output_id].iterrows():
                j += 1
                print (str(j)+") input_id:", row1.tx_id, "UTXO_id:", row1.output_id, "owner_id:", row1.sig_id, "value:",row2.value)
        print ("money_out:")
        j = 0
        for _, row1 in outputs.loc[outputs['tx_id']==row.tx_id].iterrows():
            j += 1
            print (str(j)+") output_id:", row1.id, "payee_id:", row1.pk_id, "value:", row1.value)

In [12]:
answer_1_4()

------------------- 1.4 -------------------
1. double spend:
id: 12820 tx_id: 12152 sig_id: 7941 output_id: 7998
spent in:
id: 8666 tx_id: 8231 sig_id: 7941 output_id: 7998

2. double spend:
id: 33114 tx_id: 30446 sig_id: 21807 output_id: 21928
spent in:
id: 33113 tx_id: 30446 sig_id: 21807 output_id: 21928

3. double spend:
id: 76750 tx_id: 61845 sig_id: 138980 output_id: 65403
spent in:
id: 76747 tx_id: 61843 sig_id: 138980 output_id: 65403

4. double spend:
id: 279609 tx_id: 207365 sig_id: 163625 output_id: 249860
spent in:
id: 275614 tx_id: 204751 sig_id: 163625 output_id: 249860

5. output > input:
tx_id: 100929 money_in: 5000000000 money_out: 5000000010
money_in:
1) input_id: 100929 UTXO_id: 37553 owner_id: 37334 value: 5000000000
money_out:
1) output_id: 118119 payee_id: 60205 value: 2500000000
2) output_id: 118120 payee_id: 142642 value: 2500000010


## 2 Clustering
### generate cluster dataframe

In [179]:
def cluster_entities_by_multi_input_heuristic():
    # get unique dataframe with columns = ['tx_id','sig_id']
    clusters = inputs.drop_duplicates(['tx_id','sig_id'])[['tx_id','sig_id']]
    # set all sig_id to have default cluster_id = 0
    clusters['cluster_id'] = 0
    # init max_cluster_id = 0, max_cluster_id += 1 when assigned
    max_cluster_id = 0
    # init a dict to store tx_id ever appeared, key: tx_id, value: cluster_id
    appeared_tx_id_dict = {}
    # init a dict to store sig_id ever appeared, key: sig_id, value: cluster_id
    appeared_sig_id_dict = {}
    # traverse all rows
    # 'appeared' means: it has already been assinged a cluster_id
    # pk_id == sig_id
    for _, row in clusters.iterrows():
        if row.tx_id not in appeared_tx_id_dict.keys():
            if row.sig_id in appeared_sig_id_dict.keys():
                # if signature id appeared but not transaction id,
                # we add {tx_id: cluster_id} to dict and set cluster_id of current row
                appeared_tx_id_dict[row.tx_id] = appeared_sig_id_dict[row.sig_id]
                row.cluster_id = appeared_sig_id_dict[row.sig_id]
            else:
                # if neither appeared,
                # we need to check whether there were other signature appeared, in the same transaction
                for _, row1 in clusters.loc[clusters['tx_id']==row.tx_id].iterrows():
                    if row1.sig_id in appeared_sig_id_dict.keys():
                        # if other signature S appeared in the same transaction,
                        # then we set cluster_id of current row to S's cluster_id
                        # and add {tx_id: cluster_id}, {sig_id: cluster_id} to dict respectively
                        appeared_tx_id_dict[row.tx_id] = appeared_sig_id_dict[row1.sig_id]
                        appeared_sig_id_dict[row.sig_id] = appeared_sig_id_dict[row1.sig_id]
                        row.cluster_id = appeared_sig_id_dict[row1.sig_id]
                        break
                if row.cluster_id == 0:
                    # if all signature id in the same transaction not appeared before,
                    # we assign to a new cluster_id and update the dict
                    max_cluster_id += 1
                    appeared_tx_id_dict[row.tx_id] = max_cluster_id
                    appeared_sig_id_dict[row.sig_id] = max_cluster_id
                    row.cluster_id = max_cluster_id
        else:
            # if tx_id appeared before, set the row cluster_id to corresponding value (from the dict)
            # and update the dict
            row.cluster_id = appeared_tx_id_dict[row.tx_id]
            if row.sig_id in appeared_sig_id_dict.keys():
                pass
            else:
                appeared_sig_id_dict[row.sig_id] = appeared_tx_id_dict[row.tx_id]
    return clusters

In [276]:
clusters = cluster_entities_by_multi_input_heuristic()
clusters.columns = ['tx_id','pk_id','cluster_id']

### 2.1 Specific cluster

In [277]:
def answer_2_1():
    print ("------------------- 2.1 -------------------")
    print ("How big is the cluster that contains public key (pk_id) 41442?")
    cluster_id_41442 = next(clusters.loc[clusters['pk_id']==41442].iterrows())[1].cluster_id
    print (clusters['cluster_id'].value_counts()[cluster_id_41442])
    print ("Identify the cluster according to its keys with the lowest and highest numeric values.")
    print ("lowest pk_id:",clusters.loc[clusters['cluster_id']==cluster_id_41442,'pk_id'].min())
    print ("highest pk_id:",clusters.loc[clusters['cluster_id']==cluster_id_41442,'pk_id'].max())    

In [278]:
answer_2_1()

------------------- 2.1 -------------------
How big is the cluster that contains public key (pk_id) 41442?
50
Identify the cluster according to its keys with the lowest and highest numeric values.
lowest pk_id: 40284
highest pk_id: 41911


### 2.2 Biggest cluster

In [279]:
def answer_2_2():
    print ("------------------- 2.2 -------------------")
    print ("Which cluster has the largest number of keys, and how many keys does it contain?")
    temp = clusters.drop_duplicates('pk_id')
    cluster_id, number_of_keys = temp['cluster_id'].value_counts().idxmax(), temp['cluster_id'].value_counts().max()
    print ("number of keys:",number_of_keys)
    print ("Identify it according to its keys with the lowest and highest numeric values.")
    print ("lowest pk_id:",temp.loc[temp['cluster_id']==cluster_id,'pk_id'].min())
    print ("highest pk_id:",temp.loc[temp['cluster_id']==cluster_id,'pk_id'].max())

In [280]:
answer_2_2()

------------------- 2.2 -------------------
Which cluster has the largest number of keys, and how many keys does it contain?
number of keys: 901
Identify it according to its keys with the lowest and highest numeric values.
lowest pk_id: 2172
highest pk_id: 89688


### 2.3 Richest cluster

In [315]:
def answer_2_3():
    print ("------------------- 2.3 -------------------")
    print ("As of the last block in the dataset, which cluster controls the most unspent bitcoins, and how many bitcoins does it control? Again, identify it according to its keys with the lowest and highest numeric values.")
    temp_clusters = pandas.merge(clusters,UTXOs[['pk_id','value']],on='pk_id')
    temp_clusters.columns = ['tx_id','pk_id','cluster_id','UTXO']
    temp = temp_clusters.groupby('cluster_id')['UTXO'].sum()
    print ("highest unspent bitcoin:",temp.max())
    print ("lowest pk_id:",clusters.loc[clusters['cluster_id']==cluster_id,'pk_id'].min())
    print ("highest pk_id:",clusters.loc[clusters['cluster_id']==cluster_id,'pk_id'].max())
    print ("Which transaction is responsible for sending the largest number of bitcoins to this entity (i.e., to one or more of the keys in this cluster)?")
    temp = clusters.loc[clusters['cluster_id']==cluster_id]
    temp = outputs[(outputs['pk_id'].isin(temp['pk_id'])&(~outputs['id'].isin(inputs['output_id'])))]['value']
    print(temp.sum())

In [316]:
answer_2_3()

------------------- 2.3 -------------------
As of the last block in the dataset, which cluster controls the most unspent bitcoins, and how many bitcoins does it control? Again, identify it according to its keys with the lowest and highest numeric values.
highest unspent bitcoin: 34540000000000
lowest pk_id: 108956
highest pk_id: 150019
Which transaction is responsible for sending the largest number of bitcoins to this entity (i.e., to one or more of the keys in this cluster)?
1570000000000


In [283]:
temp = clusters.groupby('cluster_id')['value'].sum()

KeyError: 'Column not found: value'

In [275]:
temp.idxmax()

31689